In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    XLMRobertaTokenizerFast,
    Trainer,
)
from datasets import load_dataset
import numpy as np
import kagglehub
import pandas as pd

2025-05-07 03:22:54.580556: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746588174.767642      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746588174.823646      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
contradictory_my_dear_watson_path = kagglehub.competition_download('contradictory-my-dear-watson')
dataset = load_dataset(contradictory_my_dear_watson_path)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
model_id = "Woodsii/FacebookAI_roberta-large-mnli"

tokenizer = XLMRobertaTokenizerFast.from_pretrained('roberta-large-mnli')
model     = AutoModelForSequenceClassification.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'XLMRobertaTokenizerFast'.


config.json:   0%|          | 0.00/893 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [4]:
from datasets import ClassLabel

# --- NEW Cell 7: tokenize + cast + rename + format ---
def tokenize_function(example):
    return tokenizer(example['premise'],
                     example['hypothesis'],
                     truncation=True,
                     padding='max_length')

# 1) Tokenize all splits
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 2) For each split, cast & rename
for split in tokenized_datasets:
    tokenized_datasets[split] = (
        tokenized_datasets[split]
        # convert float→int data + update schema
        .cast_column("label", ClassLabel(num_classes=3))
        # rename to the Trainer’s expected key
        .rename_column("label", "labels")
    )

# Now bind Torch format exactly once
tokenized_datasets.set_format(
    "torch",
    columns=["input_ids", "attention_mask", "labels"]
)

Map:   0%|          | 0/12120 [00:00<?, ? examples/s]

Map:   0%|          | 0/5195 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/12120 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5195 [00:00<?, ? examples/s]

In [5]:
test_ds = tokenized_datasets['test']

In [6]:
from torch.utils.data import DataLoader
import torch

dl = DataLoader(test_ds, batch_size=64)
model.to('cuda').eval()

all_preds = []
with torch.no_grad():
    for batch in dl:
        input_ids      = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        logits = model(input_ids, attention_mask=attention_mask).logits
        preds  = torch.argmax(logits, dim=-1)  # on GPU
        all_preds.extend(preds.cpu().numpy())

preds = np.array(all_preds, dtype=np.int64)

In [7]:
ids = tokenized_datasets['test']['id']

df = pd.DataFrame({
    "id": ids,
    "prediction": preds
})

df.to_csv("submission.csv", index=False)
print("Wrote", len(df), "rows to submission.csv")

Wrote 5195 rows to submission.csv
